## Сравнение лемматизаторов 

"Стандарты" были взяты из https://drive.google.com/drive/folders/0B600DBw1ZmZAd3lhNHQ1MUVoRkk 

In [68]:
import pandas as pd
from nltk.stem.wordnet import WordNetLemmatizer
import re
import pymorphy2
from pymystem3 import Mystem
import maru
from nltk.corpus import stopwords
import numpy as np
import glob
from nltk import word_tokenize
%matplotlib inline


%load_ext autoreload
%autoreload 2

ru_stop = stopwords.words('russian')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Сравниваются лемматизаторы:
* WordNetLemmatizer
* pymorphy2
* Mystem
* maru c rnn



In [92]:
wnl = WordNetLemmatizer()
morph = pymorphy2.MorphAnalyzer()
mystem = Mystem() 
analyzer = maru.get_analyzer(tagger='rnn', lemmatizer='pymorphy')

def tokenize(text):
    text = word_tokenize(text.lower().strip())
    return ''.join(text)
    
def wordnet_lemmatize(text):
    return wnl.lemmatize(text)

def pymorphy_lemmatize(text):

    return morph.parse(text)[0].normal_form
def mystem_lemmatize(text):

    return mystem.lemmatize(text)[0]
def mystem_lemmatize_sents(text):
    #mystem=Mystem()
    text=''.join(mystem.lemmatize(text))
    text=word_tokenize(text)
    return text
def maru_rnn_lemmatize(x):
    analyzed = analyzer.analyze([x]) 
    for i in analyzed:
        obj = i
    return obj.lemma

def evalute(gold_standart, prepared_text):
    
    return sum(gold_standart==prepared_text) / len(gold_standart)
    

Все стандарты соединены в один, удалены все знаки пуктуации. 

In [113]:
columns = ['index', 'text', 'standart_text', 'col1', 'col2']
standart = []
files = glob.glob('/proj_news_viz/data/*.txt')
for file in files:
    standart.append(pd.read_csv(file, sep = '\t', header=None, names = columns))
standart = pd.concat(standart, ignore_index=True)
standart = standart[['text', 'standart_text']]  
standart.shape         

(19560, 2)

In [114]:
standart = standart.drop(standart[standart.text.apply(lambda x: x.isalpha()==False)].index)
standart = standart.reset_index()
standart['tokenized_text'] = standart.text.apply(lambda x: tokenize(x))
standart['standart_text'] = standart.standart_text.apply(lambda x: x.lower())


Сравнение лемматизаторов и времени выполнения: 

In [117]:
lemmatizers = [maru_rnn_lemmatize, wordnet_lemmatize, pymorphy_lemmatize, mystem_lemmatize]
for fun in lemmatizers:
    start = np.datetime64('now')
    standart[fun.__name__] = standart.tokenized_text.apply(lambda x: fun(x))
    print(f'{fun.__name__}' + 
          f' accuracy: {np.round(evalute(standart.standart_text, 
                                         standart[fun.__name__]),3)}')
    print(f'Running time: {str(np.datetime64("now") - start)}')                            
        

maru_rnn_lemmatize accuracy: 0.919
Running time: 92 seconds
wordnet_lemmatize accuracy: 0.478
Running time: 0 seconds
pymorphy_lemmatize accuracy: 0.908
Running time: 1 seconds
mystem_lemmatize accuracy: 0.863
Running time: 2 seconds


In [118]:
start = np.datetime64('now')
standart['mystem_lemmatize_sents'] = pd.Series(mystem_lemmatize_sents(" ".join(standart.text)))
print('mystem_lemmatize_sents' +
      f' accuracy: {np.round(evalute(standart.standart_text,
                                     standart["mystem_lemmatize_sents"]),3)}')
print(f'Running time: {str(np.datetime64("now") - start)}')
    

mystem_lemmatize_sents accuracy: 0.878
Running time: 2 seconds


Пример предложения, обработанного разными лемматизаторами: 

In [137]:
standart[2014:2030]

,index,text,standart_text,tokenized_text,maru_rnn_lemmatize,wordnet_lemmatize,pymorphy_lemmatize,mystem_lemmatize,mystem_lemmatize_sents
2014,2573,Но,но,но,но,но,но,но,но
2015,2574,самым,самый,самым,самый,самым,самый,самый,самый
2016,2575,надежным,надёжный,надежным,надёжный,надежным,надёжный,надежный,надежный
2017,2576,способом,способ,способом,способ,способом,способ,способ,способ
2018,2577,пустить,пустить,пустить,пустить,пустить,пустить,пускать,пускать
2019,2578,под,под,под,под,под,под,под,под
2020,2579,откос,откос,откос,откос,откос,откос,откос,откос
2021,2580,дневной,дневной,дневной,дневной,дневной,дневный,дневной,дневной
2022,2581,распорядок,распорядок,распорядок,распорядок,распорядок,распорядок,распорядок,распорядок
2023,2582,было,быть,было,быть,было,быть,быть,быть
